In [240]:
from mesa import Agent
from mesa import Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

from mesa.time import RandomActivation
import numpy as np
import pandas as pd
import math

import random
random.seed(67890)

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [241]:
class Box(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)


In [242]:
class StackBox(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.isFull = False
        self.count = 0

    def addBox(self):
        self.count += 1
        if self.count == 5:
            self.isFull = True

In [243]:
# Robots
class Robot(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.hasBox = False
        self.destination = None

    def step(self):
        # Si no tenemos caja y la celda actual no es caja
        if not self.hasBox and not self.orderBox():
            self.moveRandom()
        # Si recogemos una caja
        elif self.orderBox():
            pass
        # Si tenemos caja
        elif self.hasBox:
            self.move2stack()

    def moveRandom(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=False, include_center=False
        )
        new_position = self.random.choice(possible_steps)

        # Checar que la nueva posición no esté ocupada
        cellmates = self.model.grid.get_cell_list_contents([new_position])
        if len(cellmates) > 0:
            for c in cellmates:
                if isinstance(c, Robot):
                    self.model.grid.move_agent(self, self.pos)
                    return

        self.model.grid.move_agent(self, new_position)

    def move2stack(self):
        # Si el robot esta cargando una caja, encontrar e ir a la pila más cercana
        nearest_stack = None
        min_distance = float("inf")
        for agent in self.model.schedule.agents:
            if isinstance(agent, StackBox) and not agent.isFull:
                distance = math.dist(self.pos, agent.pos)
                if distance < min_distance:
                    min_distance = distance
                    nearest_stack = agent

        # If a stack is found, set it as the destination
        if nearest_stack is not None:
            self.destination = nearest_stack.pos
            self.move_towards_destination()

    def move_towards_destination(self):
        if self.destination is None:
            return
        x, y = self.pos
        dest_x, dest_y = self.destination

        dx = 1 if dest_x > x else -1 if dest_x < x else 0
        dy = 1 if dest_y > y else -1 if dest_y < y else 0

        new_x = x + dx
        new_y = y + dy

        self.pos = (new_x, new_y)

    def orderBox(self):
        # Si la celda actual tiene una caja, recogerla
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 0:
            for c in cellmates:
                if isinstance(c, Box):
                    self.model.grid.remove_agent(c)
                    self.hasBox = True
                    return True
        return False

    def updateStackBox(self, x, y):
        for agent in self.model.schedule.agents:
            if isinstance(agent, StackBox) and agent.pos == (x, y) and not agent.isFull:
                agent.count += 1
                if agent.count == 5:
                    agent.isFull = True

    def checkStackBoxState(self):
        for agent in self.model.schedule.agents:
            if isinstance(agent, StackBox) and agent.count >= 5:
                agent.isFull = True


In [244]:
def get_grid(model):
    grid = np.zeros((model.grid.width, model.grid.height))
    for content, (x, y) in model.grid.coord_iter():
        if content == None:
            grid[x][y] = 0
        elif isinstance(content, Robot):
            grid[x][y] = 1
        # elif isinstance(content, Box):
        #     grid[x][y] = 1
        # elif isinstance(content, StackBox):
        #     if content.count == 0:
        #         grid[x][y] = 2
        #     elif content.count == 1:
        #         grid[x][y] = 3
        #     elif content.count == 2:
        #         grid[x][y] = 4
        #     elif content.count == 3:
        #         grid[x][y] = 5
        #     elif content.count == 4:
        #         grid[x][y] = 6
        #     elif content.isFull:
        #         grid[x][y] = 7
    return grid

In [245]:
class OrderingRobotsModel(Model):
    def __init__(self):
        self.num_robots = 5
        self.num_stacks = random.randint(1, 10)
        self.width = 20
        self.height = 20

        self.grid = MultiGrid(self.width, self.height, False)
        self.schedule = RandomActivation(self)
        self.running = True
        self.datacollector = DataCollector(model_reporters={"Grid": get_grid})

        # Create Robots
        robotId = 0
        for _ in range(self.num_robots):
            # Place in random cell
            robot = Robot(robotId, self)
            x, y = self.random_empty_cell()
            self.grid.place_agent(robot, (x, y))
            self.schedule.add(robot)
            robotId += 1

        # Create Stacks
        stackId = 10
        for _ in range(self.num_stacks):
            stack = StackBox(stackId, self)
            x, y = self.random_empty_cell()
            self.schedule.add(stack)
            self.grid.place_agent(stack, (x, y))
            stackId += 1

        # Distribute Boxes
        boxId = 100
        for _ in range(200):
            available_stacks = [stack for stack in self.schedule.agents if isinstance(stack, StackBox) and not stack.isFull]
            if random.choice([True, False]) and available_stacks:
                stack = random.choice(available_stacks)
                stack.addBox()

                box = Box(boxId, self)
                self.grid.place_agent(box, stack.pos)
                self.schedule.add(box)

            else:
                box = Box(boxId, self)
                x, y = self.random_empty_cell()
                self.grid.place_agent(box, (x, y))
                self.schedule.add(box)
            boxId += 1

    def random_empty_cell(self):
        empty_cells = [(x, y) for x in range(self.width) for y in range(self.height) if self.grid.is_cell_empty((x, y))]
        if not empty_cells:
            raise Exception("No empty cells available.")
        return random.choice(empty_cells)

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


In [246]:
model = OrderingRobotsModel()
for i in range(500):
    model.step()

In [247]:
grid = np.zeros((model.grid.width, model.grid.height))
for agent in model.schedule.agents:
    if agent is not None and agent.pos is not None:
        if isinstance(agent, Box):
            grid[agent.pos[0]][agent.pos[1]] = 1
        elif isinstance(agent, Robot):
            grid[agent.pos[0]][agent.pos[1]] = 2
        elif isinstance(agent, StackBox):
            grid[agent.pos[0]][agent.pos[1]] = 3